# Generate UMAP coordinates for each plate

## Import libraries

In [1]:
import glob
import pathlib
import pandas as pd
import umap

from pycytominer import feature_select
from pycytominer.cyto_utils import infer_cp_features


## Set constants

In [2]:
# Set constants
umap_random_seed = 0
umap_n_components = 2

output_dir = pathlib.Path("results")
output_dir.mkdir(parents=True, exist_ok=True)


## Create list of paths to feature selected data per plate

In [3]:
# Set input paths
data_dir = pathlib.Path("../../../3.processing_features/data/single_cell_profiles/")

# Select only the feature selected files
file_suffix = "*sc_feature_selected.parquet"

# Obtain file paths for all feature selected plates
fs_files = glob.glob(f"{data_dir}/{file_suffix}")
fs_files


['../../../3.processing_features/data/single_cell_profiles/Plate_1_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_3_prime_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_4_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_5_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_2_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_3_sc_feature_selected.parquet']

In [4]:
# Load feature data into a dictionary, keyed on plate name
cp_dfs = {x.split("/")[-1]: pd.read_parquet(x) for x in fs_files}

# Print out useful information about each dataset
print(cp_dfs.keys())
[cp_dfs[x].shape for x in cp_dfs]


dict_keys(['Plate_1_sc_feature_selected.parquet', 'Plate_3_prime_sc_feature_selected.parquet', 'Plate_4_sc_feature_selected.parquet', 'Plate_5_sc_feature_selected.parquet', 'Plate_2_sc_feature_selected.parquet', 'Plate_3_sc_feature_selected.parquet'])


[(241, 849),
 (5506, 1146),
 (7308, 1163),
 (5793, 1174),
 (1714, 856),
 (11286, 1171)]

## Generate UMAP coordinates for each plate

**Note:** Only metadata that is common between plates are included in final data frame.

In [5]:
desired_columns = ["Metadata_Plate","Metadata_Well", "Metadata_Site", "Metadata_number_of_singlecells", "Metadata_genotype"]

# Fit UMAP features per dataset and save
for plate in cp_dfs:
    plate_name = pathlib.Path(plate).stem
    print("UMAP embeddings being generated for", plate_name)

     # Make sure to reinitialize UMAP instance per plate
    umap_fit = umap.UMAP(
        random_state=umap_random_seed,
        n_components=umap_n_components
    )

    # Make sure NA columns have been removed
    cp_df = cp_dfs[plate]
    cp_df = feature_select(
        cp_df,
        operation="drop_na_columns",
        na_cutoff=0
    )

    # Make sure that the Plate_3_prime has correct name in Metadata_Plate column
    if plate_name.replace("_sc_feature_selected", "") == "Plate_3_prime":
        cp_df["Metadata_Plate"] = "Plate_3_prime"

    # Remove rows with genotype HET for Plate_5
    if plate_name.replace("_sc_feature_selected", "") == "Plate_5":
        cp_df = cp_df[cp_df["Metadata_genotype"] != "HET"]
    
    # Process cp_df to separate features and metadata
    cp_features = infer_cp_features(cp_df)
    meta_features = infer_cp_features(cp_df, metadata=True)
    filtered_meta_features = [feature for feature in meta_features if feature in desired_columns]
    
    # Fit UMAP and convert to pandas DataFrame
    embeddings = pd.DataFrame(
        umap_fit.fit_transform(cp_df.loc[:, cp_features]),
        columns=[f"UMAP{x}" for x in range(0, umap_n_components)]
    )
    print(embeddings.shape)
    
    # Combine with metadata
    cp_umap_with_metadata_df = pd.concat([
        cp_df.loc[:, filtered_meta_features].reset_index(drop=True),
        embeddings
    ], axis=1)
    
    # randomize the rows of the dataframe to plot the order of the data evenly
    cp_umap_with_metadata_df = cp_umap_with_metadata_df.sample(frac=1, random_state=0)

    # Generate output file and save
    output_umap_file = pathlib.Path(output_dir, f"UMAP_{plate_name}.tsv")
    cp_umap_with_metadata_df.to_csv(output_umap_file, index=False, sep="\t")


UMAP embeddings being generated for Plate_1_sc_feature_selected


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(241, 2)
UMAP embeddings being generated for Plate_3_prime_sc_feature_selected


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(5506, 2)
UMAP embeddings being generated for Plate_4_sc_feature_selected


...

(5793, 2)
UMAP embeddings being generated for Plate_2_sc_feature_selected


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(1714, 2)
UMAP embeddings being generated for Plate_3_sc_feature_selected


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(11286, 2)


In [6]:
# Print an example output file
print(cp_umap_with_metadata_df.shape)
cp_umap_with_metadata_df.head()


(11286, 7)


,Metadata_Well,Metadata_Site,Metadata_number_of_singlecells,Metadata_genotype,Metadata_Plate,UMAP0,UMAP1
4136,D4,8,636,WT,Plate_3,8.818437,4.097239
5457,B10,24,86,Null,Plate_3,9.012610,6.500041
10346,F12,17,647,Null,Plate_3,8.104309,5.912871
3701,D4,1,636,WT,Plate_3,10.596231,3.488836
9596,E12,14,607,Null,Plate_3,9.566523,3.041514


In [7]:
# Set input paths
data_dir = pathlib.Path("../../../3.processing_features/data/single_cell_profiles/")

# Select only the feature selected files
fs_suffix = "*sc_feature_selected.parquet"

# Obtain file paths for all feature selected plates
fs_files = glob.glob(f"{data_dir}/{fs_suffix}")
fs_files

['../../../3.processing_features/data/single_cell_profiles/Plate_1_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_3_prime_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_4_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_5_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_2_sc_feature_selected.parquet',
 '../../../3.processing_features/data/single_cell_profiles/Plate_3_sc_feature_selected.parquet']

In [8]:
# Select file paths for plates 5, 3, and 3 prime only
selected_plates = ["Plate_5", "Plate_3", "Plate_3_prime", "Plate_4"]

# Filter and concatenate the selected plates
selected_dfs = []
for file_path in fs_files:
    plate_name = pathlib.Path(file_path).stem.replace("_sc_feature_selected", "")

    # Only read in selected plates
    if plate_name in selected_plates:
        df = pd.read_parquet(file_path)

        # Update Metadata_Plate for Plate_3_prime
        if plate_name == "Plate_3_prime":
            df["Metadata_Plate"] = "Plate_3_prime"

        # For Plate_4, only include rows where Metadata_siRNA is "None"
        if plate_name == "Plate_4":
            # Fill NaN values in "Metadata_siRNA" column with "No Construct"
            df["Metadata_siRNA"].fillna("No Construct", inplace=True)

            # Filter out rows where "Metadata_siRNA" is "No Construct"
            df = df[df["Metadata_siRNA"] == "No Construct"]

        selected_dfs.append(df)

In [9]:
# Get the column names of all DataFrames in selected_dfs
column_sets = [set(df.columns) for df in selected_dfs]

# Find the common column names across all DataFrames
common_columns = list(set.intersection(*column_sets))

len(common_columns)

869

In [10]:
# Filter each DataFrame in selected_dfs to include only common columns
selected_dfs_filtered = [df.loc[:, common_columns] for df in selected_dfs]

# Concatenate the filtered dataframes along the rows
concatenated_df = pd.concat(selected_dfs_filtered, ignore_index=True)

# Save the concatenated dataframe to a file
output_concatenated_file = pathlib.Path(output_dir, "concatenated_norm_fs_plates_5_3_3prime_4controls.parquet")
concatenated_df.to_parquet(output_concatenated_file, index=False)

print(concatenated_df.shape)
concatenated_df.head()


(24237, 869)


,Nuclei_AreaShape_Zernike_7_7,Cells_RadialDistribution_ZernikeMagnitude_GFP_9_5,Cytoplasm_RadialDistribution_ZernikePhase_DAPI_1_1,Cells_RadialDistribution_MeanFrac_CY5_1of4,Cytoplasm_AreaShape_MinorAxisLength,Nuclei_AreaShape_Zernike_3_3,Nuclei_RadialDistribution_MeanFrac_DAPI_4of4,Cytoplasm_RadialDistribution_ZernikePhase_DAPI_6_2,Nuclei_RadialDistribution_ZernikePhase_GFP_7_5,Cytoplasm_AreaShape_Zernike_7_3,...,Nuclei_RadialDistribution_ZernikeMagnitude_CY5_7_3,Nuclei_RadialDistribution_ZernikeMagnitude_RFP_6_4,Cytoplasm_Texture_Contrast_DAPI_3_03_256,Cytoplasm_RadialDistribution_ZernikeMagnitude_CY5_7_3,Cytoplasm_RadialDistribution_ZernikeMagnitude_RFP_5_1,Cytoplasm_RadialDistribution_ZernikePhase_DAPI_7_7,Cytoplasm_RadialDistribution_ZernikePhase_CY5_4_2,Cytoplasm_Texture_Contrast_DAPI_3_02_256,Cells_RadialDistribution_ZernikePhase_CY5_9_1,Nuclei_RadialDistribution_ZernikePhase_GFP_5_1
0,0.820799,0.886213,1.348157,-0.043321,-0.640512,0.009005,0.599211,1.624052,-1.530704,-0.029108,...,-0.922932,0.648051,-0.328925,-0.950616,4.891929,0.117734,-0.439511,-0.380740,-1.558524,1.116379
1,-0.459939,0.662163,0.906750,-0.466150,0.712005,-0.707298,0.279534,0.938638,-1.332616,-1.394503,...,-0.708675,-0.086966,0.112074,-0.695485,1.971569,1.673029,-0.780734,0.172885,-0.798598,-0.660811
2,-0.549263,-0.981008,-1.228950,-0.587076,2.409091,0.329947,0.128703,0.165950,-1.309550,-1.444137,...,-0.597158,-0.214997,-0.351740,-1.068902,1.203431,0.881731,-1.279213,-0.508852,-1.627737,-1.645463
3,0.160538,-0.592948,1.347058,0.651058,0.305211,0.383156,0.583868,-1.710975,0.426957,-0.628029,...,3.074006,0.061657,-0.019988,-0.606614,-0.858070,-0.666867,0.204312,0.077855,-1.514506,-1.695278
4,-0.891624,-0.212319,-0.605177,2.561400,0.265424,-0.361653,-0.101686,1.506895,0.311295,-0.275872,...,-0.383712,-1.134100,-0.559689,-0.702972,-0.723711,0.992172,0.016732,-0.242222,-0.612905,-0.854758


In [11]:
desired_columns = ["Metadata_Plate","Metadata_Well", "Metadata_Site", "Metadata_number_of_singlecells", "Metadata_genotype"]

# Make sure to reinitialize UMAP instance
umap_fit = umap.UMAP(
    random_state=umap_random_seed,
    n_components=umap_n_components
)

# Process cp_df to separate features and metadata
cp_features = infer_cp_features(concatenated_df)
meta_features = infer_cp_features(concatenated_df, metadata=True)
filtered_meta_features = [feature for feature in meta_features if feature in desired_columns]

# Fit UMAP and convert to pandas DataFrame
embeddings = pd.DataFrame(
    umap_fit.fit_transform(concatenated_df.loc[:, cp_features]),
    columns=[f"UMAP{x}" for x in range(0, umap_n_components)]
)
print(embeddings.shape)

# Combine with metadata
cp_umap_with_metadata_df = pd.concat([
    concatenated_df.loc[:, filtered_meta_features],
    embeddings
], axis=1)

# randomize the rows of the dataframe to plot the order of the data evenly
cp_umap_with_metadata_df = cp_umap_with_metadata_df.sample(frac=1, random_state=0)

# Generate output file and save
output_umap_file = pathlib.Path(output_dir, f"UMAP_concat_model_plates_sc_feature_selected.tsv")
cp_umap_with_metadata_df.to_csv(output_umap_file, index=False, sep="\t")


/home/jenna/mambaforge/envs/python_analysis_nf1/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(24237, 2)
